In [1]:
from pypbc import *
import hashlib
import time

In [2]:
stored_params = """type a
q 8780710799663312522437781984754049815806883199414208211028653399266475630880222957078625179422662221423155858769582317459277713367317481324925129998224791
h 12016012264891146079388821366740534204802954401251311822919615131047207289359704531102844802183906537786776
r 730750818665451621361119245571504901405976559617
exp2 159
exp1 107
sign1 1
sign0 1
"""

params = Parameters(param_string=stored_params)
pairing = Pairing(params)

In [3]:
def setup():
    g = Element.random(pairing, G1)
    h = Element.random(pairing, G1)
    zeta = Element.random(pairing, G2)
    
    return g, h, zeta

g, h, zeta = setup()
zeta

026E58BBCE8840F9D2487A48E3D44C923345266698499D88EA4F8FCE7F3C63EFA5ADDACF467BA3A66CD340CF0AE88891CF51E36657E1B54EB209C907480A5A53F9

In [4]:
def init(num, tg):
    t_m, m = [], []
    for i in range(num):
        m.append(Element(pairing, GT))
        t_m.append(Element.random(pairing, G2))
        
    m = [pairing.apply(tg, t_m[i]) for i in range(num)]
    
    return m

m_a = init(10, g)
m_a

[(0x31085F6CAB4443DCB1D27814912AF3D431CB3F7C042BB9A4E00AB5A2D71CF7B9B722807EA85459D440452A1B1B11DCF0FB48F53D3052146AB7E3E1D4CDE86186, 0xA1D8ADE5A88B0F3338689796788CB4401EEDF17554CCE9E7D8566B82625FB30B28A469A14B9856F332783937B2965AABB5BEC6028B02D7B9C8D94E58452BC53D),
 (0x71B810F785C1CE8C66A2CEEAA7443BDF9A941DCCD4897AF79E2FC80110439F9376203851D7AD40661405E2846A5DA50ADB8B8F2308927FF8A6A7987A6A9ABA6E, 0x7B25CBF267895138D87EAD8D7BD36AFD821F87338D0D4DFF14CC326EC13AEB986838D82A58E145D9F6F28606A0706809BE27308F0E0B611CCDA9C83DBA3A5429),
 (0x192080F6A8D1F8B921A95637501664AB20C87B470522D7213AC36B98818D3C4749126A55BF1BB37F91F5177EA5A7EA0DF74275222F0676C5D1DAAF0BED3B1C8D, 0x35BDD1483B250E7B8C2129A70132E7459853D2C630B867AD1D0EA53B5F2767480475F1090023D59773167565F8A09CA42D789A7373BF6A4AB4FC2AF5CEBFF559),
 (0x96EBDD38FCD2653E5986C1FC20AC35B24AFCCF963E2D5CEC3CB6328E75018AD5E020630279656CBB673E75D0A2E4389420C682C0539757D338F52B2C6EC07957, 0x7B3420403A75140A08E405878A9B479699F925AD2BF3B327152D7EFA4A6CEC1

In [5]:
def dkg(t_zeta):
    dpk = Element(pairing, G2)
    
    alpha = Element.random(pairing, Zr)
    dpk = t_zeta**alpha
    
    return dpk, alpha

dpk, alpha = dkg(zeta)
alpha

0x071272022616C2C30B884BC73C4BCE768710BEFA

In [8]:
def ukg(tg, th):
    sk, pk = [], []
    for i in range(6):
        sk.append(Element.random(pairing, Zr))

    pk = [(tg**sk[i]) * (th**sk[i+1]) for i in range(0, 6, 2)]
    
    return sk, pk

sk1, pk1 = ukg(g, h)
sk2, pk2 = ukg(g, h)
sk1

[0x28C35331B7181F3D1A4DE0EC1F31B317F258F04C,
 0x6550506965AEC2AF0534D48402B7EED4CAC180A4,
 0x04CC6521A5E77C3AE438621DB8762C07F3AFE46E,
 0x06BFFF6BF21AC80CBC8500C6BFCCAC22C5D22475,
 0x2827DB875CA8572E769A61E758A592263D7D89DB,
 0x7EED720C0F7D9F35E9339828EB9DCD3CB8BC8D03]

In [9]:
def tkg(tdpk, tsk):
    tk = []
    for i in range(6):
        tk.append(Element(pairing, G2))
    
    tk = [tdpk**tsk[i] for i in range(6)]
    
    return tk

tk = tkg(dpk, sk1)
tk

[026D69EFE2108CBB2C2F8C415E0E1BE82EB7ED3D5BC886E234D7BE63C1DF37BF1D73B7080C09F707FBAE6C2CE7F2E18093C2D3E6B946E387FF3026922E97743812,
 024505F8E9F3645CA0B591CE89CC924AE3EB3BAA36C7FF8B1F0DE5488A6A2D9ED8E114532F12D2DA92A35E220EEE50EF6416D0EC6D09775220FE922776A21F394F,
 037B73112D239EA568B97F3771F3E5EDC988271E0F41DAF0414DF51A08C013C4CC5F769B61A69379BC2FDA9EDB661E50806069787BAED06CD56AADD76A87D78C54,
 0390087E1776620D1252EFE94B6214C13BC501288348CD249DEE7665459119CD29A4BB29938C3B863962E8753E197F39DB4519D5D0CD79D7D468A4B44D2FEC3F70,
 038EA73811715E9EE3D0B481A3B3B99B2228384036E280F4F54C647D72877E0E5DAE925BF9EF158F0126856A571C1803B0474634DBD756883BBF6537F258DD61F2,
 031002F7A31440FD699EE52D09E77611B2FDD9002E8293BE6AAA7B66C190CE8EAC4C898E47A00B8F5F1A0D88302E7C932CA644A4775B96F3FFC913AED5BD7F2801]

In [10]:
sk1, pk1 = ukg(g, h)
tk1 = tkg(dpk, sk1)

sk2, pk2 = ukg(g, h)
tk2 = tkg(dpk, sk2)
tk1

[023F93F80B9BC398796606584D30A04187F899ADACBF3628F8F880EAD1623910A93EAA18551E73B2A662076CB81BFED1BF22F8ABBF676C2F56990F0727FC95A2B4,
 0377B7CEA02F8892E0ECA6A557DDD1816DEBDA6C3BBB642D644077317B578362F44D3CA892276A4FAF44B7968A9CA8FCE27D837AEE99ABD8B5C1D69859DD13B294,
 0274EAB40E72E9745DEB6924745339FFCD909BCAE8DC94ED533F0913E8444A51F8361F6761EF9624D7B508CE6489953DED2D23C01ECCEEA97352AE517F9AF030BB,
 0319D934D596A2841BC982BB58D61E099FAFECC47853D2BA1FC1FFCCA17BB6791490C80889B58D5662E0668CC264419516374D88FF99596188D4230FAEA4B28BFD,
 02A05A52F30F295D9D69469ED53F7DF4A19426A726081ED28FED00D744A6E6AFF26CBB811DEA65114A0C6C6A90654CD2D8422E6356FDC5746A5639D058D7C859D7,
 039020603784E9CD24FD0690169CE4032836A9D2F9EDECE1D452FB6796E0A1880E3C20573EC011C937D45A115B5A62D881F76C73F19469E8E5109B02961E7A8C7F]

In [11]:
def get_hash(c1, c2, c3):
    h_in = str(c1) + str(c2) + str(c3)
    sha512 = hashlib.sha512()
    sha512.update(h_in.encode('utf8'))
    hv = sha512.hexdigest()
    ev = Element.from_hash(pairing, Zr, hv)
    
    return ev

r = Element.random(pairing, Zr)
        
w1 = g**r
w2 = h**r

x = Element(pairing, GT)
x = pairing.apply(pk1[0], zeta)

theta = get_hash(w1, w2, x)
theta

0x3930393333663766623333353662306332643961

In [16]:
def enc(num, tg, th, t_zeta, tpk, ttk, tsk, tm):
    one = Element.one(pairing, Zr)
    start = time.time()
    c = []
    for i in range(num):
        r = Element.random(pairing, Zr)
        
        w1 = tg**r
        w2 = th**r

        e = pairing.apply(tpk[0]**r, zeta)
        x = (pairing.apply(tpk[0]**r, zeta)) * tm[i] # e((g^s * h^t)^r, zeta)
        
        print((e**(-one)*x) == tm[i])
        
        e1 = pairing.apply(w1, ttk[0])
        e2 = pairing.apply(w2, ttk[1])
        
        theta = get_hash(w1, w2, x)
        y = pairing.apply((tpk[1]**r) * (tpk[2]**theta)**r, zeta)
        
        e3 = pairing.apply(w1, ttk[2])
        e4 = pairing.apply(w2, ttk[3])
        e5 = pairing.apply(w1, ttk[4])
        e6 = pairing.apply(w2, ttk[5])
        e7 = e3 * e4 * (e5**theta) * (e6**theta)
#         print((y**alpha) == e7)
        
        c.append([w1, w2, x, y])
    
    print("Enc Time: ", time.time() - start)
    return c

st = time.time()
c1 = enc(10, g, h, zeta, pk1, tk1, sk1, m_a)
dec_time = (time.time()) / 10
c2 = enc(10, g, h, zeta, pk2, tk2, sk2, m_a)
print('enc time: ', dec_time)

True
True
True
True
True
True
True
True
True
True
Enc Time:  0.41677379608154297
True
True
True
True
True
True
True
True
True
True
Enc Time:  0.42031097412109375
enc time:  156920952.654201


In [204]:
one = Element.one(pairing, Zr)
print((c1[0][2] * (c2[0][2]**-one))**alpha)
print((c1[0][3] * (c2[0][3]**-one)))

(0x27A7A280F3C22946AD32EA7314977E1F5B33990CD13CC4FDA9C840A93ACC074556411FD9A7574D0E9F81DDF27B1D169EF07B52E4CCA294BF0D29F4C9F409637E, 0x22CA5752B43340B33505DE142BE4E31816C1A846E109EEB012D12444A0DAA208F33ED8ED26974C2900767C88E7668C16B3C047FA1E4B11829E368A17C9908FEC)
(0x81BBD7D6A29321FAB6BFC947CCAA1A828C17FEAD24DD7BDAAE22D868E64B6B9D509B5CB27AA4B7E627880080B596F68550322A4876FF4542BD0E81E20B24B269, 0x4789ABC8CC3598821B6D67D56BB2D4FA232D576E1B8DC7EBCC756E9DC8485D578E11EEFEDAFE0EBDD078F7E8B367EE471002C5EB7AE301E800AD9563977AEC34)


In [15]:
def dec(tsk, tc):
    m = []
    one = Element.one(pairing, Zr)
    
    for i in range(len(tc)):
        w1 = tc[i][0]
        w2 = tc[i][1]
        x = tc[i][2]
        y = tc[i][3]
        
        theta = get_hash(w1, w2, x)
        ty1 = (w1**(tsk[2] + theta*tsk[4]))
        ty2 = (w2**(tsk[3] + theta*tsk[5]))
        e1 = pairing.apply((ty1 * ty2), zeta)
        
        if y != e1:
            return None
        else:
            tx = (w1**tsk[0]) * (w2**tsk[1])
            e2 = pairing.apply(tx, zeta)
            m.append(x * (e2**(-one)))
#             print(m)
    
    return m

st = time.time()
m_b = dec(sk1, c1)
dec_time = (time.time() - st) / len(m_b)
print('dec time: ', dec_time)
for i in range(len(m_a)):
    print(m_a[i] == m_b[i])

dec time:  0.02130310535430908
True
True
True
True
True
True
True
True
True
True


In [211]:
def IVgen(tc1, tc2, ttk1, ttk2):
    v = []
    count = 0
    one = Element.one(pairing, Zr)
    
    for i in range(len(c1)):
        for j in range(len(c2)):
            print(i, j)
            
            omega1 = Element.random(pairing, Zr)
            omega2 = Element.random(pairing, Zr)
        
            w1_1 = tc1[i][0]
            w2_1 = tc1[i][1]
            x_1 = tc1[i][2]
            y_1 = tc1[i][3]
        
            w1_2 = tc2[j][0]
            w2_2 = tc2[j][1]
            x_2 = tc2[j][2]
            y_2 = tc2[j][3]
    
            v1 = (x_1 * (x_2**(-one)))**omega1
        
            e1 = pairing.apply(w1_1, ttk1[0])
            e2 = pairing.apply(w2_1, ttk1[1])
            e3 = pairing.apply(w1_2, ttk2[0])
            e4 = pairing.apply(w2_2, ttk2[1])
        
            v2 = ((e1 * e2) * ((e3 * e4)**(-one)))**omega1
        
            print("x: ", (v1**alpha) == v2)
            if (v1**alpha) == v2:
                count += 1
        
            v3 = (y_1 * (y_2**(-one)))**omega2
        
            theta1 = get_hash(w1_1, w2_1, x_1)
            theta2 = get_hash(w1_2, w2_2, x_2)
        
            e5 = pairing.apply(w1_1, ttk1[2])
            e6 = pairing.apply(w2_1, ttk1[3])
            e7 = (pairing.apply(w1_1, ttk1[4]))
            e8 = (pairing.apply(w2_1, ttk1[5]))
            temp_y1 = e5 * e6 * (e7**theta1) * (e8**theta1)
        
            e9 = pairing.apply(w1_2, ttk2[2])
            e10 = pairing.apply(w2_2, ttk2[3])
            e11 = (pairing.apply(w1_2, ttk2[4]))
            e12 = (pairing.apply(w2_2, ttk2[5]))
            temp_y2 = e9 * e10 * (e11**theta2) * (e12**theta2)
        
            print("y: ", (y_2**alpha) == temp_y2)
        
            v4 = (temp_y1 * (temp_y2**(-one)))**omega2
            v.append([v1, v2, v3, v4])
    print("count: ", count)
    return v

v = IVgen(c1, c2, tk1, tk2)
print(len(v))

0 0
x:  True
y:  True
0 1
x:  False
y:  True
0 2
x:  False
y:  True
0 3
x:  False
y:  True
0 4
x:  False
y:  True
0 5
x:  False
y:  True
0 6
x:  False
y:  True
0 7
x:  False
y:  True
0 8
x:  False
y:  True
0 9
x:  False
y:  True
1 0
x:  False
y:  True
1 1
x:  True
y:  True
1 2
x:  False
y:  True
1 3
x:  False
y:  True
1 4
x:  False
y:  True
1 5
x:  False
y:  True
1 6
x:  False
y:  True
1 7
x:  False
y:  True
1 8
x:  False
y:  True
1 9
x:  False
y:  True
2 0
x:  False
y:  True
2 1
x:  False
y:  True
2 2
x:  True
y:  True
2 3
x:  False
y:  True
2 4
x:  False
y:  True
2 5
x:  False
y:  True
2 6
x:  False
y:  True
2 7
x:  False
y:  True
2 8
x:  False
y:  True
2 9
x:  False
y:  True
3 0
x:  False
y:  True
3 1
x:  False
y:  True
3 2
x:  False
y:  True
3 3
x:  True
y:  True
3 4
x:  False
y:  True
3 5
x:  False
y:  True
3 6
x:  False
y:  True
3 7
x:  False
y:  True
3 8
x:  False
y:  True
3 9
x:  False
y:  True
4 0
x:  False
y:  True
4 1
x:  False
y:  True
4 2
x:  False
y:  True
4 3
x:  False
y

In [134]:
def pTest(num, tc1, tc2, tpk1, tpk2, ttk1, ttk2, tdpk):
    v = []
    start = time.time()

    for i in range(num):
        e1 = pairing.apply(tpk1[0], tdpk)
        e2 = pairing.apply(g, ttk1[0])
        e3 = pairing.apply(h, ttk1[1])
        b1 = (e1 == (e2 * e3))
        
        e4 = pairing.apply(tpk1[1], tdpk)
        e5 = pairing.apply(g, ttk1[2])
        e6 = pairing.apply(h, ttk1[3])
        b2 = (e4 == (e5 * e6))
        
        e7 = pairing.apply(tpk1[2], tdpk)
        e8 = pairing.apply(g, ttk1[4])
        e9 = pairing.apply(h, ttk1[5])
        b3 = (e7 == (e8 * e9))
        
        e10 = pairing.apply(tpk2[0], tdpk)
        e11 = pairing.apply(g, ttk2[0])
        e12 = pairing.apply(h, ttk2[1])
        b4 = (e10 == (e11 * e12))
        
        e13 = pairing.apply(tpk2[1], tdpk)
        e14 = pairing.apply(g, ttk2[2])
        e15 = pairing.apply(h, ttk2[3])
        b5 = (e13 == (e14 * e15))
        
        e16 = pairing.apply(tpk2[2], tdpk)
        e17 = pairing.apply(g, ttk2[4])
        e18 = pairing.apply(h, ttk2[5])
        b6 = (e16 == (e17 * e18))
        
        b7 = (b1 and b2 and b3 and b4 and b5 and b6)
        if b7:
            IVgen(tc1, tc2, ttk1, ttk2)
    
pTest(10, c1, c2, pk1, pk2, tk1, tk2, dpk)

True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True


In [212]:
def dTest(tv, t_alpha):
    if((tv[2]**t_alpha) != tv[3]):
        return None
    else:
        if((tv[0]**t_alpha) != tv[1]):
            return 0
        else:
            return 1

# print(v)
for i in range(len(v)):
    print(dTest(v[i], alpha))

1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1


In [13]:
json_file = './discriminator.json'
with open(json_file, 'r') as f:
    d_info = json.load(f)

dsk_s = d_info['dsk']
dsk_v = int(dsk_s, 16)
dsk = Element(pairing, Zr, value=dsk_v)
dsk

0x568BFFE8FD340F772F08C6D2DBA013E9E14787B4